In [23]:
#DB
import os
import psycopg2

# Helpers
import numpy as np
# import qgrid
from tqdm import tqdm_notebook
import time
import pandas as pd

# Evaluation
from gap import define_k

# Plots
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pickle
import base64
import numpy as np
from tokenizer import create_bag_of_words
from clustering import Clustering
from itertools import combinations
from sklearn.metrics.cluster import normalized_mutual_info_score
from vectorizer import NCutVectorizer


## Connect to DB

In [21]:
connection = psycopg2.connect(user = "machineteaching",
                                  password = "***REMOVED***",
                                  host = "localhost",
#                                   port = "5432",
                                  database = "machineteaching")
connection.autocommit=True
cursor = connection.cursor()

In [4]:
def get_where_items(exp_id):
    cols = ["vectorizer", "min_df", "is_binary", "distance", "method", "dataset", "k", "model", "X"]
    query = "SELECT %s from EXPERIMENTS_STABILITY where experiment_id = %s" % (", ".join(cols), exp_id) 
    cursor.execute(query)
    where_items = cursor.fetchall()
    return where_items

def analyze(solutions, where_items, exp_id):
    v = eval(where_items[0][0])
    m = where_items[0][1]
    b = where_items[0][2]
    dist = where_items[0][3]
    method = where_items[0][4]
    k = where_items[0][6]
    model_db = pickle.loads(base64.b64decode(where_items[0][7]))
    X = np.asarray(where_items[0][8])

    train_data_features, vectorizer, feature_names = create_bag_of_words(solutions, v, binary=b, min_df=m)
    clustering = Clustering(train_data_features, k, metric=dist)
    clustering.seed = model_db.random_state
    
    model, document_topic, word_topic = getattr(clustering, method)()
    
#     savefig='problems_20190101_%s_%s_exp_%s_%s_document_topic_distribution' % (method, clustering.seed, 
#                                                                                    exp_id, clustering.k)
#     clustering.plot_topic_distribution(title="Topic distribution per document", ylabel="Document ID", savefig=savefig, cmap='Blues')

#     print("Count per class:")
#     clusters = clustering.document_topic.argmax(axis=1)
#     counts = np.unique(clusters, return_counts=True)
#     print(counts)
    return clustering, method, feature_names, model

In [6]:
## Cleaning database
last_id = 132
problems = Problem.objects.filter(id__gt=last_id)
# solutions_obj = Solution.objects.filter(problem__in=problems).update(ignore=True)
print("Problems to be ignored: %d" % problems.count())

problems = Problem.objects.filter(id__lte=last_id)
# problems = Problem.objects.all()
print("Problems to be used: %d" % problems.count())

solutions_obj = Solution.objects.filter(problem__in=problems, ignore=False).order_by('id')
# solutions_obj = Solution.objects.all().order_by('id')
print("Solutions to be used: %d" % solutions_obj.count())

docs_id = []
questions = []
solutions = []

# Fill separated structures
for sol in solutions_obj:
    docs_id.append(sol.id)
    questions.append(sol.problem.content)
    solutions.append(sol.content)

print("Got %d documents" %(solutions_obj.count()))

Problems to be ignored: 591
Problems to be used: 132
Solutions to be used: 54
Got 54 documents


In [25]:
%%time

vectorizers = [
    'CountVectorizer',
    'TfidfVectorizer', 
    'NCutVectorizer'
]
min_df = np.arange(0.05, 0.51, 0.05)
binary = [
    True, 
    False
]
cluster_methods = [
    'nmf',
    'lda'
]
#, 'hierarchical', 'gaussian_mixture', 'spectral_clustering']
metric = [
    'euclidean', 
#     'cosine',
#     'correlation'
]

total = 0
total_rows = 0

# Grid search
for k in tqdm_notebook(range(2,16), desc="k", leave=False):
    for v in tqdm_notebook(vectorizers, desc="vectorizer", leave=False):
        for m in tqdm_notebook(min_df, desc="min_df", leave=False):
            for b in tqdm_notebook(binary, desc="binary", leave=False):
                for dist in tqdm_notebook(metric, desc="metric", leave=False):
                    for method in tqdm_notebook(cluster_methods, desc="method", leave=False):

                        query = "SELECT experiment_id from EXPERIMENTS_STABILITY where "
                        where_clause = [("vectorizer", v), ("min_df", m), ("is_binary", b), 
                                        ("distance", dist), ("method", method), ("k", k)]
                        query += " AND ".join(["%s = '%s'" % item for item in where_clause])
                        cursor.execute(query)
                        r = cursor.fetchall()

                        clusters = []
                        for exp_id in r:
                            where_items = get_where_items(exp_id[0])
#                             print("Conditions")
#                             print(where_items[0][0:7])

                            clustering, method, feature_names, model = analyze(solutions, where_items, exp_id)
                            clusters.append(clustering.document_topic.argmax(axis=1))
                    
                        # Calculate NMI
                        scores = []
                        ij = combinations(range(len(clusters)), 2)
                        for i, j in ij:
                            scores.append(normalized_mutual_info_score(clusters[i], clusters[j], 
                                                                       average_method='arithmetic'))
                            
                        scores_array = np.asarray(scores)
                        mean = scores_array.mean()
                        std = scores_array.std()
                        median = np.median(scores_array)
                        q75, q25 = np.percentile(scores_array, [75, 25])
                        iqr = q75 - q25
                        
                        cols = ["vectorizer", "is_binary", "min_df", "distance", "k", "method", "samples", 
                                "mean", "std", "median", "iqr"]
                        insert_query = "INSERT INTO EXPERIMENTS_STABILITY_STATISTICS (%s) VALUES " % ", ".join(cols)
                        values = [v, b, m, dist, k, method, scores, mean, std, median, iqr]
                        insert_query += "(" + ", ".join(["%s"]*len(values)) + ")"
                        query = cursor.mogrify(insert_query, values)
                        cursor.execute(query)

CPU times: user 1h 6min 36s, sys: 23min 47s, total: 1h 30min 23s
Wall time: 1h 37min 24s


In [51]:
query = "select * from EXPERIMENTS_STABILITY_STATISTICS"
cursor.execute(query)
r = cursor.fetchall()
cols = ["vectorizer", "is_binary", "min_df", "distance", "k", "method", "samples", 
        "mean", "std", "median", "iqr"]
df = pd.DataFrame(r, columns=['id'] + cols)
cols.remove("samples")
df_nodup = df.drop_duplicates(cols)
df

,id,vectorizer,is_binary,min_df,distance,k,method,samples,mean,std,median,iqr
0,1,CountVectorizer,True,0.05,euclidean,2,nmf,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.000000,0.000000,1.000000,0.000000
1,2,CountVectorizer,True,0.05,euclidean,2,lda,"[0.031273, 0.323024, 0.262192, 0.110647, 0.222...",0.103001,0.118406,0.065195,0.160914
2,3,CountVectorizer,False,0.05,euclidean,2,nmf,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.000000,0.000000,1.000000,0.000000
3,4,CountVectorizer,False,0.05,euclidean,2,lda,"[0.172722, 0.0155707, 0.0174887, 0.0274318, 0....",0.095874,0.131942,0.047567,0.123678
4,5,CountVectorizer,True,0.10,euclidean,2,nmf,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.000000,0.000000,1.000000,0.000000
5,6,CountVectorizer,True,0.10,euclidean,2,lda,"[8.21291e-16, 0.0134417, 0.0453586, 0.675619, ...",0.268771,0.289673,0.177901,0.491427
6,7,CountVectorizer,False,0.10,euclidean,2,nmf,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.000000,0.000000,1.000000,0.000000
7,8,CountVectorizer,False,0.10,euclidean,2,lda,"[0.0636377, 0.0353419, 0.0166534, 0.101486, 0....",0.315411,0.252067,0.275020,0.492630
8,9,CountVectorizer,True,0.15,euclidean,2,nmf,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.000000,0.000000,1.000000,0.000000
9,10,CountVectorizer,True,0.15,euclidean,2,lda,"[0.743163, 0.452991, 0.452991, 0.639032, 0.639...",0.530120,0.311876,0.452991,0.463055


In [52]:
df_nodup

,id,vectorizer,is_binary,min_df,distance,k,method,samples,mean,std,median,iqr
0,1,CountVectorizer,True,0.05,euclidean,2,nmf,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.000000,0.000000,1.000000,0.000000
1,2,CountVectorizer,True,0.05,euclidean,2,lda,"[0.031273, 0.323024, 0.262192, 0.110647, 0.222...",0.103001,0.118406,0.065195,0.160914
2,3,CountVectorizer,False,0.05,euclidean,2,nmf,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.000000,0.000000,1.000000,0.000000
3,4,CountVectorizer,False,0.05,euclidean,2,lda,"[0.172722, 0.0155707, 0.0174887, 0.0274318, 0....",0.095874,0.131942,0.047567,0.123678
4,5,CountVectorizer,True,0.10,euclidean,2,nmf,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.000000,0.000000,1.000000,0.000000
5,6,CountVectorizer,True,0.10,euclidean,2,lda,"[8.21291e-16, 0.0134417, 0.0453586, 0.675619, ...",0.268771,0.289673,0.177901,0.491427
6,7,CountVectorizer,False,0.10,euclidean,2,nmf,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.000000,0.000000,1.000000,0.000000
7,8,CountVectorizer,False,0.10,euclidean,2,lda,"[0.0636377, 0.0353419, 0.0166534, 0.101486, 0....",0.315411,0.252067,0.275020,0.492630
8,9,CountVectorizer,True,0.15,euclidean,2,nmf,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.000000,0.000000,1.000000,0.000000
9,10,CountVectorizer,True,0.15,euclidean,2,lda,"[0.743163, 0.452991, 0.452991, 0.639032, 0.639...",0.530120,0.311876,0.452991,0.463055


In [58]:
df_nodup[(df_nodup["method"] == "lda") & (df_nodup["k"] == 12)].sort_values("mean", ascending=False)

,id,vectorizer,is_binary,min_df,distance,k,method,samples,mean,std,median,iqr
1372,1373,CountVectorizer,True,0.50,euclidean,12,lda,"[0.904176, 0.646643, 0.511686, 0.960021, 0.587...",0.724260,0.141824,0.710577,0.230690
1412,1413,TfidfVectorizer,True,0.50,euclidean,12,lda,"[0.471922, 0.922232, 0.420578, 0.779354, 0.914...",0.705132,0.185971,0.740782,0.346240
1448,1449,NCutVectorizer,True,0.45,euclidean,12,lda,"[1.0, 0.339234, 0.339234, 0.339234, 0.339234, ...",0.677949,0.293299,0.722527,0.660766
1368,1369,CountVectorizer,True,0.45,euclidean,12,lda,"[0.641965, 0.645576, 0.795778, 0.633135, 0.724...",0.626641,0.109348,0.631339,0.134477
1408,1409,TfidfVectorizer,True,0.45,euclidean,12,lda,"[0.614767, 0.726834, 0.722208, 0.599547, 0.507...",0.616361,0.181397,0.630189,0.188415
1360,1361,CountVectorizer,True,0.35,euclidean,12,lda,"[0.58197, 0.59562, 0.600357, 0.501735, 0.45270...",0.615479,0.082304,0.604864,0.083529
1348,1349,CountVectorizer,True,0.20,euclidean,12,lda,"[0.509166, 0.615974, 0.632939, 0.710226, 0.680...",0.613605,0.080672,0.626517,0.119396
1356,1357,CountVectorizer,True,0.30,euclidean,12,lda,"[0.662835, 0.71471, 0.615584, 0.588335, 0.6887...",0.607921,0.099905,0.607320,0.133391
1352,1353,CountVectorizer,True,0.25,euclidean,12,lda,"[0.627802, 0.556637, 0.566303, 0.503856, 0.582...",0.594315,0.064022,0.593855,0.079024
1344,1345,CountVectorizer,True,0.15,euclidean,12,lda,"[0.818765, 0.628815, 0.747116, 0.551453, 0.585...",0.592825,0.113017,0.595732,0.120331


In [ ]:
df_nodup[(df_nodup["method"] == "lda") & (df_nodup["k"] == 12)].sort_values("mean", ascending=False)